<a href="https://colab.research.google.com/github/Geringer13/Netology_pyda/blob/master/22_Python_Feature_engineering_%D0%98%D1%81%D0%BF%D0%BE%D0%BB%D1%8C%D0%B7%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5_pandas_%D0%B8_numpy_%D0%B4%D0%BB%D1%8F_%D0%BE%D1%87%D0%B8%D1%81%D1%82%D0%BA%D0%B8_%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85_Homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from sklearn.datasets import load_boston
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
import matplotlib.ticker as ticker
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import math

data = pd.read_csv(load_boston()['filename'], skiprows=1)

        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pupil-teacher ratio by town
        - B        1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
        - LSTAT    % lower status of the population
        - MEDV     Median value of owner-occupied homes in $1000's

CRIM - уровень преступности на душу населения по городам

ZN - доля жилой земли, зонированной для участков более 25 000 кв футов

INDUS - доля не-розничных бизнес-акров на город

CHAS - Charles River (= 1, если тракт ограничивает реку; 0 в противном случае)

NOX - концентрация оксидов азота NOX (частей на 10 миллионов)

RM - среднее количество комнат на жилое помещение

AGE - ВОЗРАСТНАЯ доля занимаемых владельцами квартир, построенных до 1940 года

DIS - взвешивает расстояния до пяти бостонских центров занятости

RAD - индекс доступности радиальных магистралей

TAX - Налоговая ставка на имущество на 10 000

PLRATIO - Соотношение учеников и учителей PTRATIO по городам

B - доля чернокожих по городам.
LSTAT - % ниже статуса населения

MEDV - Медианная стоимость домов, занимаемых владельцами, в $ 1000



In [0]:
data.head()

Итак, разделим датафрейм на две части, первая часть все показатели без таргета которая часть только показатель MEDV, который мы будем предсказывать. Обучим модель и посмотрим ошибку на не подготовленных данных.

In [0]:
data_reg = data.drop(['MEDV'], axis=1)

In [0]:
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression

X = data_reg
y = data['MEDV']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(X)
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Все дальнейшие действия будут напралены на очистку и подготовку данных для обучения модели и сравнения ошибок.

In [0]:
data

In [0]:
print(len(data))
print(data.shape)

In [0]:
data.info()

In [0]:
data.isna().sum()

In [0]:
data.describe()

In [0]:
sns.pairplot(data)

In [0]:
sns.heatmap(data.corr(),annot=True,cmap='RdYlGn',linewidths=0.2) 
plt.show()

Судя по матрице корреляций существует сильная корреляция между RM и таргетом, попробуем посчитать ошибку модели на одном этом показателе. Мы видим, что ошибка стала выше, поэтому будем работать с данными дальше.

In [0]:
data_rm = data['RM']

In [0]:
data_rm = data_rm.apply(math.ceil).astype(int)

In [0]:
data_rm = np.array(data_rm).reshape(-1, 1)

In [0]:
X = data_rm
y = data['MEDV']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(X)
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Выпишем столбцы, которые можно убрать из анализа

'CRIM' - ok - аномалий и выбросов не замечено

'ZN' - delete - аномалий и выбросов не замечено

'INDUS' - mb - аномалий и выбросов не замечено

'CHAS' - delete - аномалий и выбросов не замечено

'NOX' - mb - аномалий и выбросов не замечено 

'RM' - ok - нужно округлить в большую сторону

'AGE' - delete

'DIS' - mb -  аномалий и выбросов не замечено

'RAD' - mb

'TAX' - ok

'PTRATIO' - delete

'B' - ok

'LSTAT' - ок - обратная полиномиальная регрессия

'MEDV'



Анализируем распределения и анномалии по каждому параметру

In [0]:
data_reg['RM'] = data_reg['RM'].apply(math.ceil).astype(int)

In [0]:
data['MEDV'].hist()

Итак, проанализировав данные напрашивается несколько вопросов, во первых странные данные по ценам ниже 10, уровень ТАХ = 666, уровень темнокожих ниже 10, а также показатель RAD = 24. Попробую два подхода: 1 - удалить эти строки, 2 - заменить на среднее.

Для начала сократим количество показателей, построим модель после удаления нескольких
столбцов, а затем нормализуем эти данные, построим модель и сравним результаты.

In [0]:
data_approach_1 = data_reg.drop(['AGE', 'PTRATIO', 'ZN', 'CHAS'], axis=1)
data_approach_1.head()

In [0]:
headers = list(data_approach_1.columns.values)

In [0]:
scaler = preprocessing.StandardScaler()
data_approach_2 = scaler.fit_transform(data_approach_1)
df_norm_1 = pd.DataFrame(data_approach_2)
df_norm_1.columns = headers
df_norm_1.head()

In [0]:
df_norm_1.columns

Поработаем со странными значениями ТАХ = 666, уровень темнокожих ниже 10, а также показатель RAD = 24

In [0]:
data_reg_1 = data[data['TAX'] !=666]

In [0]:
data_reg_1_target = data_reg_1['MEDV']

In [0]:
data_reg_1 = data_reg_1.drop(['MEDV'], axis=1)
data_reg_1.head()

In [0]:
data_reg_1[data_reg_1['MEDV'] == 50]

In [0]:
len(data_reg_1_target)

In [0]:
tax666 = data[data['TAX'] == 666]

In [0]:
sns.pairplot(tax666)

In [0]:
data_wth_666 = data[data['TAX'] != 666]

In [0]:
data_wth_666 = data_wth_666[data_wth_666['MEDV'] != 50]

In [0]:
data_wth_666.hist(figsize=(15,15))
plt.show()

In [0]:
sns.pairplot(data_wth_666)

In [0]:
data_wth_666.describe()

In [0]:
data.describe()

In [0]:
fig, ax = plt.subplots(figsize=(5,5))
ax.scatter(x=data_wth_666['RM'], y=data_wth_666['MEDV'])
plt.show()

Попробуем полиномиальную регрессию на показателе LSTAT

In [0]:
from sklearn.preprocessing import PolynomialFeatures

poly_reg = PolynomialFeatures(degree=2)

X_poly = poly_reg.fit_transform(X_train_lstat)

pol_reg = LinearRegression()
pol_reg.fit(X_poly, y_train)

pol_reg.score(poly_reg.fit_transform(X_test_lstat), y_test)

**Результаты моделей**

1.  На всех данных, без обработки
*  R2: 	 0.6197381783666025
*   RMSE: 	 4.918517322266345
*   MAE: 	 3.5835665363854448
2.  На одном показателе RM
*   R2: 	 0.504181351276971
*   RMSE: 	 5.6163513893118395
* MAE: 	 4.044859175712278
3. На одном показателе RM, после округления в бОльшую сторону
*   R2: 	 0.3851370305838009
* RMSE: 	 6.2543475658102015
* MAE: 	 4.733766124544446
4. На показателях 'CRIM', 'INDUS', 'NOX', 'RM', 'DIS', 'RAD', 'TAX', 'B', 'LSTAT'
* R2: 	 0.6613075829998756
* RMSE: 	 4.641897856718249
* MAE: 	 3.4572858241822906
5. На показателях 'CRIM', 'INDUS', 'NOX', 'RM', 'DIS', 'RAD', 'TAX', 'B', 'LSTAT' после нормализаци MixMaxScaller
* R2: 	 0.6613075829998756
* RMSE: 	 4.641897856718249
* MAE: 	 3.4572858241822906
6. На показателях 'CRIM', 'INDUS', 'NOX', 'RM', 'DIS', 'RAD', 'TAX', 'B', 'LSTAT' после нормализаци StandartScaller
* R2: 	 0.661307582999878
* RMSE: 	 4.6418978567182325
* MAE: 	 3.4572858241822724
7. После удаления строк со значением TAX = 666
* R2: 	 0.8494399850402503
* RMSE: 	 3.4873120913550344
* MAE: 	 2.7893019732589526
8. После удаления строк со значением TAX = 666, на одном показателе RM
* R2: 	 0.7305161670460163
* RMSE: 	 3.846147689313335
* MAE: 	 3.0537879697836248
9. Полиномиальная регрессия на показателе LSTAT, а также TAX != 666
* R2: 	 0.46121759657776096
* RMSE: 	 5.4383393081105
* MAE: 	 4.225968690134918
 	 










In [0]:
X = data_wth_666
y = data_wth_666['MEDV']

In [0]:
X = np.array(data_wth_666['LSTAT']).reshape(-1, 1)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=30)

In [0]:
# Попробуем полиномиальную регрессию по показателю LSTAT и TAX != 666
from sklearn.preprocessing import PolynomialFeatures
poly_reg = PolynomialFeatures(degree=2)

X_poly = poly_reg.fit_transform(X_train)

estimator = LinearRegression()
estimator.fit(X_train, y_train)

y_pred = estimator.predict(X_test)

# pol_reg.score(poly_reg.fit_transform(X_test_lstat), y_test)
print("R2: \t", r2_score(y_test, y_pred))
print("RMSE: \t", np.sqrt(mean_squared_error(y_test, y_pred)))
print("MAE: \t", mean_absolute_error(y_test, y_pred))

In [0]:
estimator = LinearRegression()
estimator.fit(X_train, y_train)

y_pred = estimator.predict(X_test)

print("R2: \t", r2_score(y_test, y_pred))
print("RMSE: \t", np.sqrt(mean_squared_error(y_test, y_pred)))
print("MAE: \t", mean_absolute_error(y_test, y_pred))

In [0]:
for a, b in zip(y, y_pred):
  print('True: {}, Pred: {}, Diff: {}'.format(a*1000, b*1000, a*1000-b*1000))